<a href="https://colab.research.google.com/github/pandeysurbhi/AspireNex/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install opendatasets

In [3]:
import opendatasets as od

**Preprocessing of the Dataset **

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

In [5]:
!pip install tensorflow opendatasets matplotlib seaborn

In [6]:

#dataset_path = r"C:\Users\tosur\Temp Folder\Folder\Projects\Major Project\brain-tumor-mri-dataset"


In [7]:
from google.colab import drive
drive.mount('/content/drive')
#dataset_path = "/content/drive/My Drive/Brain-Tumor-Dataset"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
dataset_path = "/content/drive/My Drive/brain-tumor-mri-dataset"

if os.path.exists(dataset_path):
    print("✅ Dataset folder found!")
    print("Contents:", os.listdir(dataset_path))
else:
    print("❌ Dataset folder NOT found. Check path again.")

✅ Dataset folder found!
Contents: ['pituitary', 'meningioma', 'notumor', 'glioma']


In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255,   # Normalize pixel values
    rotation_range=20, # Random rotations
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting dataset (80% train, 20% validation)
)



In [10]:
# Load training data
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),   # Resize images to match DenseNet121 input
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 5723 images belonging to 4 classes.


In [11]:
# Load validation data
valid_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 1430 images belonging to 4 classes.


In [13]:
# Load DenseNet121 (pretrained on ImageNet)
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [14]:
# Freeze the base model to keep pretrained weights
base_model.trainable = False

In [17]:
# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Reduce overfitting
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output_layer = Dense(4, activation='softmax')(x)  # 4 classes


In [18]:
# Create final model
model = Model(inputs=base_model.input, outputs=output_layer)


In [19]:
# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [20]:

# Show model summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d            │ (None, 230, 230, 3)    │              0 │ input_layer[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,408 │ zero_padding2d[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d_1          │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1 (MaxPooling2D)      │ (None, 56, 56, 64)     │              0 │ zero_padding2d_1[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_bn         │ (None, 56, 56, 64)     │            256 │ pool1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_0_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 128)    │          8,192 │ conv2_block1_0_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 128)    │            512 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 128)    │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 32)     │         36,864 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_concat       │ (None, 56, 56, 96)     │              0 │ pool1[0][0],           │
│ (Concatenate)             │                        │                │ conv2_block1_2_conv[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block2_0_bn    

 Total params: 7,300,932 (27.85 MB)

 Trainable params: 263,428 (1.00 MB)

 Non-trainable params: 7,037,504 (26.85 MB)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,  # Start with 10, then fine-tune
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 1637s 9s/step - accuracy: 0.3503 - loss: 1.7146 - val_accuracy: 0.8000 - val_loss: 0.6599
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 1452s 8s/step - accuracy: 0.5931 - loss: 0.9889 - val_accuracy: 0.8189 - val_loss: 0.5632
Epoch 3/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 1478s 8s/step - accuracy: 0.6626 - loss: 0.8284 - val_accuracy: 0.8385 - val_loss: 0.5228
Epoch 4/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 1442s 8s/step - accuracy: 0.6894 - loss: 0.7619 - val_accuracy: 0.8406 - val_loss: 0.4981
Epoch 5/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 1432s 8s/step - accuracy: 0.7291 - loss: 0.6834 - val_accuracy: 0.8538 - val_loss: 0.4683
Epoch 6/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 1419s 8s/step - accuracy: 0.7353 - loss: 0.6815 - val_accuracy: 0.8657 - val_loss: 0.4484
Epoch 7/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 1425s 8s/step - accuracy: 0.7418 - loss: 0.6615 - val_accuracy: 0.8650 - val_loss: 0.4402
Epoch 8/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 1417s 8s/step - accuracy: 0.7494 - loss: 0.6511 - 

In [ ]:
# Unfreeze base model layers
base_model.trainable = True

In [ ]:
# Recompile model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.00001),  # Smaller LR for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:

# Train again
history_finetune = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,  # Fine-tuning
    verbose=1
)

In [ ]:
# Evaluate on validation data
val_loss, val_acc = model.evaluate(valid_generator)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")


In [ ]:
# Save the trained model
model.save("/content/brain_tumor_model.h5")
print("Model saved successfully!")